##**Maestría en Inteligencia Artificial Aplicada**
###**Curso: Inteligencia Artificial y Aprendizaje Automático**
####Tecnológico de Monterrey
####Prof Luis Eduardo Falcón Morales

### **Adtividad de la Semana 8**
####**Reducción de Dimensionalidad - Descomposición en Valores Singulares (SVD) y Sistemas de Recomendación**


**Esta Actividad deberás resolverla de manera individual.**

**Nombre y matrícula:**

*   Javier Alejandro Pérez Garza
*   A01284386


In [52]:
# Agrega aquí todas las librerías y paquetes adicionales que requieras.
from google.colab import drive
drive.mount('/content/drive')
import os
import numpy as np
import pandas as pd
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics.pairwise import cosine_similarity

DIR = "/content/drive/MyDrive/Colab Notebooks/MNA/TC4030 - Inteligencia Artificial y Aprendizaje Automatico/Semana 8/"
os.chdir(DIR)

import warnings
warnings.filterwarnings('ignore')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### **Liga de Datos de la UCI "Restaurant & consumer data".**
### **Descarga "RCdata.zip" y de ahí necesitaremos los archivos "rating_final.csv" y "geoplaces2.csv".**

https://archive.ics.uci.edu/dataset/232/restaurant+consumer+data

### **Puedes también usar la siguiente liga:**

https://www.kaggle.com/datasets/prajitdatta/movielens-100k-dataset

In [53]:
# Descarga los archivos de la página de la UCI a partir de los cuales generaremos
# nuestras matrices de utilidad.

data_rating = pd.read_csv("rating_final.csv", header='infer', sep=",")
data_geo = pd.read_csv("geoplaces2.csv", header='infer',  encoding='latin-1')

print(data_rating.shape, data_geo.shape)


(1161, 5) (130, 21)


## **Ejercicio - 1**

### **Explica cuál es el propósito de usar el argumento "latin-1" al cargar el segundo archivo.**

++++++++ Inicia la sección de agregar texto: +++++++++++

El argumento encoding=`'latin-1'` se utiliza al cargar el archivo CSV para implementar el encoding ISO_8859-1:1987; es decir, asegurarse de que los caracteres especiales, como acentos y símbolos propios del español latino (ñ, ü, ¡, ç, etc) se interpreten correctamente.

++++++++ Termina la sección de agregar texto. +++++++++++

## **Ejercicio - 2**

#### **Cada restaurante debe tener un nombre diferente:**

In [54]:
# Veamos la lista de restaurantes.
# Todos ellos son restaurantes ubicados en alguna ciudad del país de México:

data_geo['name'].values

array(['Kiku Cuernavaca', 'puesto de tacos', 'El Rincón de San Francisco',
       'little pizza Emilio Portes Gil', 'carnitas_mata',
       'Restaurant los Compadres', 'Taqueria EL amigo ', 'shi ro ie',
       'Pollo_Frito_Buenos_Aires', 'la Estrella de Dimas',
       'Restaurante 75', 'Abondance Restaurante Bar',
       'El angel Restaurante', 'Restaurante Pueblo Bonito',
       'Mcdonalds Parque Tangamanga', 'Tortas y hamburguesas el gordo',
       'Sirlone', 'rockabilly ', 'Unicols Pizza', 'TACOS EL GUERO',
       'Restaurant El Muladar de Calzada', 'La Posada del Virrey',
       'Restaurant and Bar and Clothesline Carlos N Charlies', 'KFC',
       'Giovannis', 'Restaurant Oriental Express', 'Mariscos Tia Licha',
       'cafe ambar', 'Restaurante la Gran Via', 'don burguers',
       'Restaurante y Pescaderia Tampico', 'Rincon del Bife',
       'La Fontana Pizza Restaurante and Cafe',
       'Restaurante la Estrella de Dima', 'El Rincon de San Francisco',
       'Preambulo Wifi Zone 

*   ### **Todos los restaurantes que aparecen en esta lista son negocios diferentes, en el sentido de que físicamente están ubicados en diferentes ciudades de México, aunque algunos pertenezcan a una franquicia, como Vips.**

*   ### **Para los fines de este ejercicio, estaremos considerando a todos los restaurantes como diferentes. Sin embargo, en el caso de Vips, existen tres restaurantes en esta lista capturados con los nombres: VIPS, Vips y vips. Que por sus coordenadas de latitud y longitud sabemos que están ubicados en SLP, Cuernavaca y cdVictoria. Se podrían cambiar los nombres agregando la ciudad, pero como se capturaron los nombres de manera diferente con minúsculas y mayúsculas, los dejaremos por el momento de esta manera.**

*   ### **Por el momento no estaremos haciendo las recomendaciones por el lugar donde se encuentran, sino solamente por la evaluación del restaurante y otras características que no tienen que ver con la ubicación.**

*   ### **Existen otras franquicias capturadas también de manera diferente, salvo el caso de 'Gorditas Dona Tota', en el cual dos de estos restaurantes que están ubicados en Cd.Victoria fueron registrados exactamente con el mismo nombre. Para que nuestro sistema de recomendación pueda diferenciarlos, deberás cambiar el nombre de uno de ellos como se te indica a continuación.**  

### **De los dos restaurantes registrados con el nombre de 'Gorditas Dona Tota', encuentra sus índices y cambia el que tiene el mayor índice en data_geo[name] al de  'Gorditas Dona Tota 2'.**

In [55]:
# Ejercicio-2

# ************* Inlcuye aquí tu código:*****************************

indices_Dona_Tota = data_geo[data_geo['name'] == 'Gorditas Dona Tota'].index

if len(indices_Dona_Tota) > 1:
    data_geo.loc[indices_Dona_Tota.max(), 'name'] = 'Gorditas Dona Tota 2'


# *********** Aquí termina la sección de agregar código *************

print('Verificando el cambio de nombre:\n', data_geo.loc[indices_Dona_Tota, 'name'])

Verificando el cambio de nombre:
 89       Gorditas Dona Tota
118    Gorditas Dona Tota 2
Name: name, dtype: object


## **Ejercicio - 3**

In [56]:
# Al momento tenemos los siguientes DataFrames. Del primer archivo
# tenemos la evaluación general, la de la comida y la del servicio:

data_rating.head(3)

,userID,placeID,rating,food_rating,service_rating
0,U1077,135085,2,2,2
1,U1077,135038,2,2,1
2,U1077,132825,2,2,2


In [57]:
# Y del segundo archivo obtenemos información diversa de cada restaurante:

data_geo.head(2).T

,0,1
placeID,134999,132825
latitude,18.915421,22.147392
longitude,-99.184871,-100.983092
the_geom_meter,0101000020957F000088568DE356715AC138C0A525FC46...,0101000020957F00001AD016568C4858C1243261274BA5...
name,Kiku Cuernavaca,puesto de tacos
address,Revolucion,esquina santos degollado y leon guzman
city,Cuernavaca,s.l.p.
state,Morelos,s.l.p.
country,Mexico,mexico
fax,?,?


### **De cada uno de estos archivos selecciona y combina las variables adecuadas para obtener un nuevo DataFrame cuyas columnas sean el ID de usuario (userID), el ID del restaurante (placeID), la calificación general (rating) y el nombre del restaurante (name). A este nuevo DataFrame llamarlo df_combinado.**

In [58]:
# Ejercicio 3:

# ************* Inlcuye aquí tu código:*****************************

df_combinado = data_rating[['userID', 'placeID', 'rating']].merge(
    data_geo[['placeID', 'name']], on='placeID', how='left'
    )



# *********** Aquí termina la sección de agregar código *************


# Despleguemos la dimensión y los primeros renglones de este DataFrame:

print(df_combinado.shape)
df_combinado.head()

(1161, 4)


,userID,placeID,rating,name
0,U1077,135085,2,Tortas Locas Hipocampo
1,U1077,135038,2,Restaurant la Chalita
2,U1077,132825,2,puesto de tacos
3,U1077,135060,1,Restaurante Marisco Sam
4,U1068,135104,1,vips


## **Ejercicio - 4**

In [59]:
# Ejercicio 4:

#    Define la matriz de utilidad cuyos renglones sean los nombres de los
#    restaurantes, las columnas los IDs de los usuarios y las entradas la
#    evaluación general (rating). La llamaremos "UtMx_rating".


# ************* Inlcuye aquí tu código:*****************************

UtMx_rating = df_combinado.pivot_table(index='name', columns='userID', values='rating', fill_value=0)


# *********** Aquí termina la sección de agregar código *************


print('Dimensión de la matriz de Utilidad:')
print('(restaurantes, usuarios) =', (UtMx_rating.shape))
UtMx_rating.head()

Dimensión de la matriz de Utilidad:
(restaurantes, usuarios) = (130, 138)


userID,U1001,U1002,U1003,U1004,U1005,U1006,U1007,U1008,U1009,U1010,...,U1129,U1130,U1131,U1132,U1133,U1134,U1135,U1136,U1137,U1138
name,,,,,,,,,,,,,,,,,,,,,
Abondance Restaurante Bar,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Arrachela Grill,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Cabana Huasteca,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
Cafe Chaires,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Cafeteria cenidet,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### **Similaridades con la matriz obtenida de la Factorización SVD Truncada:**

*   ### **Aplicaremo la Factorización SVD con los 66 primeros valores singulares a la matriz de utilidad UtMx_rating.**

*   ### **El valor de 66 se seleccionó porque como verás, esta cantidad de valores singulares es suficiente para describir al menos el 95% de la varianza de la matriz de utilidad.**

*   ### **Recordemos que la factorización SVD de una matriz $A$ tiene la forma:** $A_{m\times n} = U_{m\times m}\Sigma_{m\times n}V_{n\times n}^T$

*   ### **Usaremos la función coseno como medida de similaridad**

*   ### **Buscaremos restaurantes similares al llamado  "tacos de barbacoa enfrente del Tec"**.

In [60]:
num_componentes=66  # componentes más significativas

# Obtengamos la matriz de factores latentes de los restaurantes:
SVD_rating = TruncatedSVD(n_components=num_componentes)  # inicializamos
mat_fact_lat_restaurantes = SVD_rating.fit_transform(UtMx_rating) # SVD truncada-ndarray

print('Dimensión Vectores latentes restaurantes:', mat_fact_lat_restaurantes.shape)
print('Dimensión Traspuesta Vectores latentes usuarios:', SVD_rating.components_.shape)

# Calculemos la variabilidad acumulada de las componentes utilizadas:
print('\nVariabilidad acumulada de las componentes utilizadas: %.3f' % SVD_rating.explained_variance_ratio_[0:num_componentes].sum())

# matriz de similaridad
sim_matrix = cosine_similarity(mat_fact_lat_restaurantes)

# Restaurante de referencia:
restaurante_de_referencia = "tacos de barbacoa enfrente del Tec"
nombres_rest = UtMx_rating.T.columns  # lista de los nombres de los restaurantes.
idx_rest = list(nombres_rest).index(restaurante_de_referencia) # índice del restaurante de referencia.
sim_vector = sim_matrix[idx_rest] # Vector de similaridad del restaurante de referencia contra todos.

# Buscando las similaridades positivas:
idx = (sim_vector>0)
mejores_sim_rate = list()
for i in range(len(nombres_rest[idx])):
  mejores_sim_rate.append((sim_vector[idx][i], nombres_rest[idx][i]))

print('\nTotal de similaridades positivas encontradas:', len(mejores_sim_rate))

# Las ordenamos de mayor a menor relevancia:
mejores_sim_rate_ordenadas = sorted(mejores_sim_rate, key=lambda x:x[0], reverse=True)

print('\nMejores recomendaciones con base al restaurante: \"%s\":' % restaurante_de_referencia)
mejores_sim_rate_ordenadas[1:11]   # omitimos el primero que es el mismo restaurante de referencia


Dimensión Vectores latentes restaurantes: (130, 66)
Dimensión Traspuesta Vectores latentes usuarios: (66, 138)

Variabilidad acumulada de las componentes utilizadas: 0.952

Total de similaridades positivas encontradas: 75

Mejores recomendaciones con base al restaurante: "tacos de barbacoa enfrente del Tec":


[(0.9312841137147382, 'vips'),
 (0.9310814507878147, 'little pizza Emilio Portes Gil'),
 (0.9271667788083259, 'tacos abi'),
 (0.5262155085407728, 'Carreton de Flautas y Migadas'),
 (0.49985802024196785, 'puesto de gorditas'),
 (0.4800290424391312, 'Taqueria EL amigo '),
 (0.35891073270218415, 'carnitas_mata'),
 (0.26519770837793827, 'Little Cesarz'),
 (0.22572481605185982, 'Gorditas Dona Tota 2'),
 (0.18662302175343323, 'carnitas mata calle Emilio Portes Gil')]

## **Ejercicio - 5**

### **Sistema Híbrido**

### **Utilicemos ahora un sistema híbrido, el cual consistirá en conjuntar la matriz de "factores_latentes_restaurantes" (que es un ndarray) obtenida previamente, con la matriz de factores adicionales (que también será un ndarray) formada por las columnas One-Hot-Encoder de los factores 'dress_code', 'accessibility', 'price' y 'franchise' del DataFrame data_geo.**

### **Por el momento no requerimos que la matriz híbrida sea un DataFrame, con que sea un arreglo de NumPy, ndarray, será suficiente.**

### **Los renglones de la matriz híbrida siguen representando a cada restaurante, pero ahora con la información de los vectores latentes de la factorización SVD, más los vectores OneHotEncoding de los nuevos factores que se incluyeron.**



In [61]:
# Ejercicio 5:

# Define el DataFrame "df_fact_adicionales" formada por los factores 'dress_code',
# 'accessibility', 'price' y 'franchise'.
# Después generar la matriz (ndarray), "mat_fact_adicionales_ohe", que se obtiene
# al tranformar las columnas (variables categóricas nominales) de "df_fact_adicionales"
# mediante OneHotEncoding.
# Finalmente conjuntar horizontalmente la matriz "mat_fact_lat_restaurantes"
# con "mat_fact_adicionales_ohe" para obtener la "matriz_hibrida" (ndarray).
# Toma en cuenta que "matriz_hibrida" estará formada por las columnas de "mat_fact_lat_restaurantes"
# seguida por las columnas de "mat_fact_adicionales_ohe".


# ************* Inlcuye aquí tu código:*****************************
data_geo = data_geo.sort_values(by='name').reset_index(drop=True)
df_fact_adicionales = data_geo[['dress_code', 'accessibility', 'price', 'franchise']]

encoder = OneHotEncoder(handle_unknown='ignore', drop='first', sparse_output=False)
mat_fact_adicionales_ohe = encoder.fit_transform(df_fact_adicionales)

matriz_hibrida = np.hstack((mat_fact_lat_restaurantes, mat_fact_adicionales_ohe))


# *********** Aquí termina la sección de agregar código *************


print('Dimensión de la matriz híbrida:')
matriz_hibrida.shape

Dimensión de la matriz híbrida:


(130, 73)

In [62]:
# Recordemos que "restaurante_de_referencia" es "tacos de barbacoa enfrente del Tec".
# Obtener el índice del restaurante
idx = data_geo[data_geo['name'] == restaurante_de_referencia].index[0]

print('Restaurante de referencia:', restaurante_de_referencia)
print('Índice del restaurante de referencia:', idx)

Restaurante de referencia: tacos de barbacoa enfrente del Tec
Índice del restaurante de referencia: 125


## **Ejercicio - 6:**

### **Una vez que tenemos la matriz híbrida, con dicha matriz deberás obtener las similitudes (con respecto a la función coseno) del restaurante de referencia con relación a todos los restaurantes, ordenarlos de mayor a menor con respecto a la similitud del coseno y desplegar los primeros 10 de mayor similitud (sin incluir al restaurante de referencia mismo).**

In [63]:
# Ejercicio 6

# ************* Inlcuye aquí tu código:*****************************
# matriz de similaridad
sim_matrix = cosine_similarity(matriz_hibrida)

# Restaurante de referencia:
restaurante_de_referencia = "tacos de barbacoa enfrente del Tec"
nombres_rest = UtMx_rating.T.columns  # lista de los nombres de los restaurantes.
idx_rest = list(nombres_rest).index(restaurante_de_referencia) # índice del restaurante de referencia.
sim_vector = sim_matrix[idx_rest] # Vector de similaridad del restaurante de referencia contra todos.

# Buscando las similaridades positivas:
idx = (sim_vector>0)
mejores_sim_rate = list()
for i in range(len(nombres_rest[idx])):
  mejores_sim_rate.append((sim_vector[idx][i], nombres_rest[idx][i]))

print('\nTotal de similaridades positivas encontradas:', len(mejores_sim_rate))

# Las ordenamos de mayor a menor relevancia:
mejores_sim_rate_ordenadas = sorted(mejores_sim_rate, key=lambda x:x[0], reverse=True)

print('\nMejores recomendaciones con base al restaurante: \"%s\":' % restaurante_de_referencia)
mejores_sim_rate_ordenadas[1:11]   # omitimos el primero que es el mismo restaurante de referencia



# *********** Aquí termina la sección de agregar código *************


Total de similaridades positivas encontradas: 125

Mejores recomendaciones con base al restaurante: "tacos de barbacoa enfrente del Tec":


[(0.961403582800622, 'tacos abi'),
 (0.876089230720682, 'little pizza Emilio Portes Gil'),
 (0.7236710592177795, 'vips'),
 (0.666533968124893, 'puesto de gorditas'),
 (0.6609940737108123, 'Carreton de Flautas y Migadas'),
 (0.6370604677173352, 'Carnitas Mata  Calle 16 de Septiembre'),
 (0.504211826530293, 'Taqueria EL amigo '),
 (0.5018946826771443, 'cafe ambar'),
 (0.49065009584469693, 'Pollo_Frito_Buenos_Aires'),
 (0.4875424887484824, 'Hamburguesas saul')]

## **Ejercicio - 7**

### **Incluye tus comentarios y conclusiones de la actividad. En particular comenta las diferencias que encuentras entre un sistema de recomendación basado solamente en la factorización SVD y uno híbrido.**

++++++++ Inicia la sección de agregar texto: +++++++++++

Los sistemas de recomendación se han vuelto parte de nuestra vida diaria. Los sistemas de streaming online, las páginas de recomendaciones de restaurantes e incluso las redes sociales, o las de busca de trabajo. Desde este punto de vista, es escencial que los profesionistas, y más si están inmersos en temas de inteligencia artificial, conozcan los algoritmos y la teoría detrás de los sistemas que influencian las decisiones de consumo y preferencias globales.

En esta actividad, se ha implementado un sistema de recomendación basado en la factorización SVD y se ha comparado con otro con un enfoque híbrido que incorpora características adicionales de los restaurantes. Mientras que el método SVD, apoyado en patrones latentes del `rating`, permitió identificar similitudes entre restaurantes basadas únicamente en el `rating` de los usuarios, el modelo híbrido mejoró las recomendaciones al incluir variables como `dress_code`, `accessibility`, `price`, `franchise`. Como resultado, encontramos que las recomendaciones del sistema híbrido presentan una mejor representación del contexto real de cada restaurante, permitiendo sugerencias más relevantes en función del restaurante de referencia.




++++++++ Termina la sección de agregar texto. +++++++++++

>> **Fin de la Actividad de Sistemas de Recomendación**